In [30]:
import numpy as np
import pandas as pd
from utils import *

In [31]:
df = load_from_txt('/Users/shamanthk/Documents/iomics/DoWhy-attribution/datasets/sachs.2005.continuous.txt')
df = df.dropna()
df.drop_duplicates(inplace=True)
labels = list(df.columns)
df.head()

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk
0,26.4,13.2,8.82,18.30,58.80,6.61,17.0,414.0,17.00,44.9,40.0
1,35.9,16.5,12.30,16.80,8.13,18.60,32.5,352.0,3.37,16.5,61.5
2,59.4,44.1,14.60,10.20,13.00,14.90,32.5,403.0,11.40,31.9,19.5
3,73.0,82.8,23.10,13.50,1.29,5.83,11.8,528.0,13.70,28.6,23.1
4,33.7,19.8,5.19,9.73,24.80,21.10,46.1,305.0,4.66,25.7,81.3


In [32]:
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(contamination=0.0005, random_state=42)
df['anomaly'] = iso_forest.fit_predict(df)
df_iso_outliers = df[df['anomaly'] == -1]

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
df_iso_outliers

,raf,mek,plc,pip2,pip3,erk,akt,pka,pkc,p38,jnk,anomaly
2776,2890.0,4451.0,698.0,1263.0,157.0,55.2,365.0,13.60,1186.0,6153.0,1134.0,-1
2821,122.0,209.0,965.0,1540.0,10.3,160.0,882.0,55.20,799.0,4105.0,2227.0,-1
2995,538.0,764.0,2267.0,3619.0,91.4,82.0,474.0,23.70,679.0,3555.0,1176.0,-1
3177,196.0,334.0,2666.0,4740.0,184.0,93.9,519.0,9.22,717.0,3555.0,3398.0,-1


In [35]:
df_iso_outliers.to_csv('anamoulous_data.csv')